In [5]:
with open('../data/minishake.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [8]:
print(len(text))

print(text[:1000])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [12]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [27]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [chars.index(c) for c in s]
decode = lambda l: ''.join([chars[i] for i in l])


In [34]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [36]:
n = int(0.9 * len(data))

train_data = data[:n]
val_data = data[n:]

In [50]:
torch.manual_seed(1337)

batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split =="train" else val_data

    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x,y


xb,yb = get_batch('train')


for ele in xb:
    print(decode(ele))

print("----")

for ele in yb:
    print(decode(ele))


Let's he
for that
nt that 
MEO:
I p
----
et's hea
or that 
t that h
EO:
I pa


In [53]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx)

        loss = F.cross_entropy(logits, targets)

        return logits
    

m = BigramLanguageModel(vocab_size)
out = m(xb,yb)
print(out.shape)

torch.Size([4, 8, 65])
